In [2]:
import pandas as pd
import numpy as np
import re

In [26]:
pattern = {
            'date': r'\bdébarqué(?:e|es|s)? (?:et passé(?:e|es|s)? aux forges (?:à|de|du|au|en)|et vendu (?:à|de|du|au|en)|à l\'hôpital à la relâche (?:à|de|du|au|en)|et resté(?:e|s|es)? employé(?:e|s|es)? (?:à|de|du|au|en)|et parti pour aller (?:à|de|du|au|en)|pour aller (?:à|de|du|au|en)|et resté(?:e|s|es)? officier des troupes (?:à|de|du|au|en)|malade à la relache (?:à|de|du|au|en)|au comptoir (?:à|de|du|au|en)|et resté(?:e|s|es)? chez son oncle (?:à|de|du|au|en)|pour l\'hôpital (?:à|de|du|au|en)|par ordre (?:à|de|du|au|en)|de gré à l\'hôpital (?:à|de|du|au|en)|et resté(?:e|es|s)? à l\'hôpital (?:à|de|du|au|en)|avec ses effets (?:à|de|du|au|en)|et resté(?:e|es|s)? à ses frais (?:à|de|du|au|en)|et resté(?:e|es|s)? malade à l\'hôpital (?:à|de|du|au|en)|et resté(?:e|es|s)? aumônier à l\'hôpital (?:à|de|du|au|en)|avec ses hardes, de gré à gré (?:à|de|du|au|en)|avec ses hardes (?:à|de|du|au|en)|ou resté(?:e|es|s)? (?:à|de|du|au|en)|et engagé(?:e|es|s)? dans la troupe (?:à|de|du|au|en)|dans la|et engagé(?:e|es|s)? dans le régiment (?:à|de|du|au|en)|et engagé(?:e|es|s)? dans|et engagé(?:e|es|s)? (?:à|de|du|au|en)|pour la traite (?:à|de|du|au|en)|et retenu (?:à|de|du|au|en)|pour maladie (?:à|de|du|au|en)|pour affaires de famille (?:à|de|du|au|en)|à malade à l\'hôpital (?:à|de|du|au|en)|et revenu sur le|malade dans le bateau du pilote (?:à|de|du|au|en)|et incorporé(?:e|es|s)? à la garnison (?:à|de|du|au|en)|et incorporé(?:e|es|s)? dans la garnison (?:à|de|du|au|en)|pour le service (?:à|de|du|au|en)|malade et resté(?:e|es|s)? (?:à|de|du|au|en)|de bonne volonté (?:à|de|du|au|en)|et mis à bord (?:à|de|du|au|en)|malade à la relâche (?:à|de|du|au|en)|à la relâche (?:à|de|du|au|en)|sans ordre (?:à|de|du|au|en)|et resté(?:e|es|s)? comme matelot (?:à|de|du|au|en)|malade à l\'hôpital du roi (?:à|de|du|au|en)|à l\'hôpital du roi (?:à|de|du|au|en)|au bas (?:à|de|du|au|en)|et (?:congédié(?:e|es|s)?|soldé(?:e|es|s)?) (?:à|de|du|au|en)|et remis sur le fort (?:à|de|du|au|en)|et remis (?:à|de|du|au|en)|pour se rendre (?:à|de|du|au|en)|blessé(?:e|s|es)? malade (?:à|de|du|au|en)|blessé(?:e|s|es)? (?:à|de|du|au|en)|de gré pour le service (?:à|de|du|au|en)|de gré à gré (?:à|de|du|au|en)|puis passé(?:e|es|s)? malade (?:à|de|du|au|en)|et resté(?:e|es|s)? (?:à|de|du|au|en)|et passé(?:e|es|s)? sur (?:le|la)|et passé(?:e|es|s)? (?:à|de|du|au|en)|et entré(?:e|es|s)? aux forges (?:à|de|du|au|en)|et mis au (?:prison|cachot) (?:à|de|du|au|en)|de son gré (?:à|de|du|au|en)|malade d\'une descente à la relâche (?:à|de|du|au|en)|malade du scorbut (?:à|de|du|au|en)|et resté(?:e|es|s)? (?:à|de|du|au|en)|dans le|et renvoyé(?:e|es|s)? (?:à|de|du|au|en)|et entré(?:e|es|s)? à l\'hôpital (?:à|de|du|au|en)|et payé(?:e|es|s)? (?:à|de|du|au|en)|de gré (?:à|de|du|au|en)|et resté(?:e|es|s)? à terre|au service du roi (?:à|de|du|au|en) |à l\'hôpital (?:à|de|du|au|en) |à |au |en échange (?:à|de|du|au|en) |furtivement (?:à|de|du|au|en) |malade après être tombé du haut mal (?:à|de|du|au|en) |malade à l\'hôpital (?:à|de|du|au|en) |malade (?:à|en|au|du|de) |malade et mort(?:e|es|s)? à l\'hôpital (?:à|de|du|au|en) |malade (?:à|de|du|au|en) |à malade (?:à|de|du|au|en) |à |au |en |de |aux |malade |pour )?[^\d\n,]*?(?:le|du) (\d{2}(?:-|/)\d{2}/\d{4})(?=(\s*---|\n|$|\s*passé(?:e|es|s)?(?:s|e|es)?|\s*échappé(?:e|s|es)?|\s*mort(?:e|es|s)?(?:e|s|es)?|\s*noyé(?:e|s|es)?))',
            'location': r'\bdébarqué(?:e|es|s)? (?:et passé(?:e|es|s)? aux forges (?:à|de|du|au|en)|et vendu (?:à|de|du|au|en)|à l\'hôpital à la relâche (?:à|de|du|au|en)|et resté(?:e|s|es)? employé(?:e|s|es)? (?:à|de|du|au|en)|et parti pour aller (?:à|de|du|au|en)|pour aller (?:à|de|du|au|en)|et resté(?:e|s|es)? officier des troupes (?:à|de|du|au|en)|malade à la relache (?:à|de|du|au|en)|au comptoir (?:à|de|du|au|en)|et resté(?:e|s|es)? chez son oncle (?:à|de|du|au|en)|pour l\'hôpital (?:à|de|du|au|en)|par ordre (?:à|de|du|au|en)|de gré à l\'hôpital (?:à|de|du|au|en)|et resté(?:e|es|s)? à l\'hôpital (?:à|de|du|au|en)|avec ses effets (?:à|de|du|au|en)|et resté(?:e|es|s)? à ses frais (?:à|de|du|au|en)|et resté(?:e|es|s)? malade à l\'hôpital (?:à|de|du|au|en)|et resté(?:e|es|s)? aumônier à l\'hôpital (?:à|de|du|au|en)|avec ses hardes, de gré à gré (?:à|de|du|au|en)|avec ses hardes (?:à|de|du|au|en)|ou resté(?:e|es|s)? (?:à|de|du|au|en)|et engagé(?:e|es|s)? dans la troupe (?:à|de|du|au|en)|dans la|et engagé(?:e|es|s)? dans le régiment (?:à|de|du|au|en)|et engagé(?:e|es|s)? dans|et engagé(?:e|es|s)? (?:à|de|du|au|en)|pour la traite (?:à|de|du|au|en)|et retenu (?:à|de|du|au|en)|pour maladie (?:à|de|du|au|en)|pour affaires de famille (?:à|de|du|au|en)|à malade à l\'hôpital (?:à|de|du|au|en)|et revenu sur le|malade dans le bateau du pilote (?:à|de|du|au|en)|et incorporé(?:e|es|s)? à la garnison (?:à|de|du|au|en)|et incorporé(?:e|es|s)? dans la garnison (?:à|de|du|au|en)|pour le service (?:à|de|du|au|en)|malade et resté(?:e|es|s)? (?:à|de|du|au|en)|de bonne volonté (?:à|de|du|au|en)|et mis à bord (?:à|de|du|au|en)|malade à la relâche (?:à|de|du|au|en)|à la relâche (?:à|de|du|au|en)|sans ordre (?:à|de|du|au|en)|et resté(?:e|es|s)? comme matelot (?:à|de|du|au|en)|malade à l\'hôpital du roi (?:à|de|du|au|en)|à l\'hôpital du roi (?:à|de|du|au|en)|au bas (?:à|de|du|au|en)|et (?:congédié(?:e|es|s)?|soldé(?:e|es|s)?) (?:à|de|du|au|en)|et remis sur le fort (?:à|de|du|au|en)|et remis (?:à|de|du|au|en)|pour se rendre (?:à|de|du|au|en)|blessé(?:e|s|es)? malade (?:à|de|du|au|en)|blessé(?:e|s|es)? (?:à|de|du|au|en)|de gré pour le service (?:à|de|du|au|en)|de gré à gré (?:à|de|du|au|en)|puis passé(?:e|es|s)? malade (?:à|de|du|au|en)|et resté(?:e|es|s)? (?:à|de|du|au|en)|et passé(?:e|es|s)? sur (?:le|la)|et passé(?:e|es|s)? (?:à|de|du|au|en)|et entré(?:e|es|s)? aux forges (?:à|de|du|au|en)|et mis au (?:prison|cachot) (?:à|de|du|au|en)|de son gré (?:à|de|du|au|en)|malade d\'une descente à la relâche (?:à|de|du|au|en)|malade du scorbut (?:à|de|du|au|en)|et resté(?:e|es|s)? (?:à|de|du|au|en)|dans le|et renvoyé(?:e|es|s)? (?:à|de|du|au|en)|et entré(?:e|es|s)? à l\'hôpital (?:à|de|du|au|en)|et payé(?:e|es|s)? (?:à|de|du|au|en)|de gré (?:à|de|du|au|en)|et resté(?:e|es|s)? à terre|au service du roi (?:à|de|du|au|en) |à l\'hôpital (?:à|de|du|au|en) |en échange (?:à|de|du|au|en) |furtivement (?:à|de|du|au|en) |malade après être tombé du haut mal (?:à|de|du|au|en) |malade à l\'hôpital (?:à|de|du|au|en) |malade et mort(?:e|es|s)? à l\'hôpital (?:à|de|du|au|en) |malade à l\'hôpital |malade (?:à|de|du|au|en) |à malade (?:à|de|du|au|en) |à malade |à |au |aux |en appareillant |en |de |du |malade |pour )?([\w\s\'-îÎ]+?)(?=( ---|---|\s+(?:le|du)\s+\d{2}-\d{2}/\d{4}|\s+(?:le|du)\s+\d{2}/\d{2}/\d{4}|en \d{4}|\d{2}/\d{2}/\d{4}|(?:le|du)\s+\d{2}/\d{2}/\d{4}|le \d{2}|avec sa femme|et passé(?:e|es|s)?|et remplacé(?:e|es|s)?|lors de|de bonne volonté|sur le|en (?:janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|décembre)|à la demande|au service|à l\'hôpital|et repassé(?:e|es|s)? à|pour|par ordre|et parti|à la relâche|de gré|malade|(?:le|du)\s+\?|\n|$|et n\'a pas|pour aller|au retour|et passé(?:e|es|s)? sur|en appareillant|et remis sur|avec|et rendu|sans ordre|le fleuve|et resté(?:e|es|s)?|comme tonnelier|avant|entre|pendant|en allant|dans les|de son|chez les|et commandé|à l\'arrivé|au soir|et engagé|solde|incommodé|revenu|chez|solde|\s*passé(?:e|es|s)?(?:s|e|es)?|échappé(?:e|s|es)?| échappé(?:e|s|es)?|mort(?:e|es|s)?(?:e|s|es)?| mort(?:e|es|s)?(?:e|s|es)?|noyé(?:e|s|es)?| noyé(?:e|s|es)?))'
        }

date_pattern = pattern['date']
location_pattern = pattern['location']

text = "débarqué et passé aux forges de Mr Herman le \n 10/06/1762"

date_match = re.search(date_pattern, re.sub(r'\s+', ' ', text.replace('\n', ' ')), flags=re.IGNORECASE)
loc_match = re.search(location_pattern, re.sub(r'\s+', ' ', text.replace('\n', ' ')), flags=re.IGNORECASE)



if date_match:
    print(date_match.group(1))
else:
    print("No date found")
if loc_match:
    print(str(loc_match.group(1)).strip())
else:
    print("No location found")

10/06/1762
Mr Herman


In [60]:
date_pattern = r'\bdébarqué(?:e|es|s)? (?:à l\'hôpital de |à |au |furtivement à |malade après être tombé du haut mal à |au désarmement |désarmement à |malade à l\'hôpital (?:de|du|à|au) |malade à |malade et mort à l\'hôpital (?:de|du) )?[^\d\n,]*?(?:le|du) (\d{2}/\d{2}/\d{4})(?=(\s*---|\n|$|\s*passé(?:s|e|es)?|\s*échappé(?:e|s|es)?|\s*mort(?:e|s|es)?|\s*noyé(?:e|s|es)?))'
location_pattern = r'\bdébarqué(?:e|es|s)? (?:à l\'hôpital de |à |au |furtivement à |malade après être tombé du haut mal à |au désarmement |désarmement à |malade à l\'hôpital (?:de|du|à|au) |malade à |malade et mort à l\'hôpital (?:de|du) )([^\d\n,]+?)(?=( ---|---|\s+(?:le|du)\s+\d{2}/\d{2}/\d{4}|\s+(?:le|du) ?|le\s+\d{2}/\d{2}/\d{4}|\n|$|\s*passé(?:s|e|es)?|échappé(?:e|s|es)?| échappé(?:e|s|es)?|mort(?:e|s|es)?| mort(?:e|s|es)?|noyé(?:e|s|es)?| noyé(?:e|s|es)?))'
for text in texts:
    if 'débarqué' in text:
#         text = "débarqué le ? --- d'autorité"
        print(text)

        # a servi
        # le lendemain
        # vaisseau

        date_match = re.search(date_pattern, re.sub(r'\s+', ' ', text.replace('\n', ' ')), flags=re.IGNORECASE)
        loc_match = re.search(location_pattern, re.sub(r'\s+', ' ', text.replace('\n', ' ')), flags=re.IGNORECASE)



        if date_match:
            print(date_match.group(1))
        else:
            print("No date found")
        if loc_match:
            print(loc_match.group(1))
        else:
            print("No location found")
        print('*********************************************')

NameError: name 'texts' is not defined